In [17]:
def hama_response(N):
    filename = 'ave1/C1--HighPulse-in-100-out1700-an2100--00000.dat'
    time,h = np.loadtxt(filename,unpack=True)
    ids = np.random.randint(1,300,size=(N-1,1))
    for id in ids:
        filename = 'ave1/C1--HighPulse-in-100-out1700-an2100--%05i.dat' % id
        time,hi = np.loadtxt(filename,unpack=True)
        h = np.row_stack((h,hi))
    return time,h

In [20]:
def gaussian_pde(time,N,rate,width):
    mu = time[3*len(time)/4]
    pde = (1/np.sqrt(width**2*np.pi*2))*np.exp(-(rate*(time[len(time)/2:]-mu)*1.0e9)**2/width)
    fig = plt.figure()
    plt.plot(time[len(time)/2:],pde)
    ids=[]
    count = 0
    for i in range(len(pde)):
        if count == N:
            break
        elif np.random.choice([1,0], 1, p=[pde[i],1-pde[i]]) == 1:
            ids = np.append(ids,len(time)/2+i)
            count +=1
    return ids

In [ ]:
%matplotlib notebook

import numpy as np
from scipy.fftpack import fft, fftfreq, ifft
import matplotlib.pyplot as plt

N = 10

time, h = hama_response(N)
havg = np.mean(h,axis=0)
H = fft(h,axis=1)
Havg = fft(havg)

freq = fftfreq(time.size,d = time[1]-time[0])

"""#random distribution of hits
index = np.random.randint(len(time)/2,len(time),size=(N,1))
"""

#gaussian probability distribution of hits
rate = 1.0   #sampling rate
width = 10.0  #gaussian width
index = gaussian_pde(time,N,rate,width)
print index

S = np.exp(-2*np.pi*1j*freq*time[index])
G = S*H

S = np.sum(S,axis=0)
Gavg = S*Havg

G = np.sum(G,axis=0)

s = np.real(ifft(S))
g = np.real(ifft(G))
gavg = np.real(ifft(Gavg))

fig = plt.figure()
ax = fig.subplots()
ax.plot(time,s,label='impulses')
ax.plot(time,g,label='signals')
ax.plot(time,gavg,label='noiseless_signals')
ax.set_title(label='Signals')
ax.grid(True)
ax.legend()

In [16]:
len(range(10))/2

5